In [22]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, auc

In [33]:
class NaiveBayes:

    def fit(self, X, y):
        n_samples, n_features = X.shape
        self._classes = np.unique(y)
        n_classes = len(self._classes)
        

        # calculate mean, var, and prior for each class
        self._mean = np.zeros((n_classes, n_features), dtype=np.float64)
        self._var = np.zeros((n_classes, n_features), dtype=np.float64)
        self._priors =  np.zeros(n_classes, dtype=np.float64)

        for idx, c in enumerate(self._classes):
            X_c = X[y==c]
            self._mean[idx, :] = X_c.mean(axis=0)
            self._var[idx, :] = X_c.var(axis=0)
            self._priors[idx] = X_c.shape[0] / float(n_samples)

    def predict(self, X):
        X = np.array(X)
        y_pred = [self._predict(x) for x in X]
        return np.array(y_pred)

    def get_parameter(self):
        print("Class Probabilties : ",self._priors)
    
    def _predict(self, x):
        posteriors = []

        # calculate posterior probability for each class
        for idx, c in enumerate(self._classes):
            prior = np.log(self._priors[idx])
            posterior = np.sum(np.log(self._pdf(idx, x)))
            posterior = prior + posterior
            posteriors.append(posterior)
            
        # return class with highest posterior probability
        return self._classes[np.argmax(posteriors)]
            

    def _pdf(self, class_idx, x):
        mean = self._mean[class_idx]
        var = self._var[class_idx]
        numerator = np.exp(- (x-mean)**2 / (2 * var))
        denominator = np.sqrt(2 * np.pi * var)
        return numerator / denominator
    
    def score(self,x_test,y_test):
        y_pred = self.predict(x_test)
        return sum([1 for i,j in zip(y_pred,y_test) if i==j]) / len(y_test)
    
#     def draw_roc(self,x_test,y_test):
#         roc_auc = dict()
#         fpr = dict()
#         tpr = dict()
#         y_score = self.predict(x_test)
#         for i in range(len(self._classes)):
#             fpr[i], tpr[i], _ = roc_curve(y_test, y_score)
#             roc_auc[i] = auc(fpr[i], tpr[i])
#         plt.plot(fpr[i], tpr[i], label='ROC curve (area = %0.2f)' % roc_auc[i])

In [34]:
df = pd.read_csv("data7.csv",header=None)
df.head()

,0,1,2,3
0,1,1,1,-1
1,1,1,2,-1
2,1,1,3,-1
3,1,1,4,-1
4,1,1,5,-1


In [35]:
df[3].unique()

array([-1,  0,  1], dtype=int64)

In [36]:
X_train, X_test, y_train, y_test = train_test_split(df[[0,1,2]],df[3],test_size=0.3)

In [37]:
model = NaiveBayes()

In [38]:
model.fit(X_train,y_train)

In [44]:
y_predicted = model.predict(X_test)

In [43]:
from sklearn.metrics import classification_report 

In [45]:
 print(classification_report(y_test,y_predicted))

              precision    recall  f1-score   support

          -1       0.85      0.65      0.73        17
           0       0.82      0.97      0.89        29
           1       1.00      0.94      0.97        18

    accuracy                           0.88        64
   macro avg       0.89      0.85      0.86        64
weighted avg       0.88      0.88      0.87        64



In [40]:
model.score(X_test,y_test)

0.875

In [41]:
model.get_parameter()

Class Probabilties :  [0.25170068 0.44217687 0.30612245]
